In [ ]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import calendar
import numpy as np
from datetime import date
from datetime import time

In [ ]:
#here you define dates (start and end) that you want to get

start_date = input('Start date in the form of: "yyyy-mm-dd": ')
end_date = input('End date in the form of: "yyyy-mm-dd": ')

start_date_time = input('Hour of the start date (24-hour clock): "hh": ')
start_date2 = start_date + ' ' + start_date_time + ':' + '00:00'

end_date_time = input('Hour of the end date (24-hour clock): "hh": ')
end_date2 = end_date + ' ' + end_date_time + ':' + '00:00'


def past_price(crypto,interval,start,end):
    
    start = calendar.timegm(datetime.fromisoformat(start).timetuple()) * 1000   #if you have date as a string in the form of 'yyyy-mm-dd', you use this to convert the string into a unix time
    end = calendar.timegm(datetime.fromisoformat(end).timetuple()) * 1000  #calendar.timegm() returns unix time, however you need to have timetuple inside brackets (see docs)
        
    url= 'https://api.binance.com/api/v3/klines?symbol=' + crypto + '&interval=' + interval + '&limit=1000'
        
    url+= '&startTime=' + str(start)
    url+= '&endTime=' + str(end)
    
    df3 = pd.read_json(url)     #you take data from API (you get raw data)
    df3.columns = ['Opentime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Closetime',
                       'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore']    #to name the columns
    
    df3.Opentime = pd.to_datetime(df3.Opentime, unit='ms')   #conversion from unix time
    df3.Closetime = pd.to_datetime(df3.Closetime, unit='ms')

    df3['Date'] = df3.Opentime.dt.strftime("%d/%m/%Y")     #to get a specified date format
    df3['Time'] = df3.Opentime.dt.strftime("%H:%M:%S")
    
    df3 = df3.drop(['Closetime', 'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote',
                      'Ignore'], axis=1)
    
    column_names = ['Opentime', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df3 = df3[column_names]     #to have this specific sequence of columns in a DataFrame
    
    return df3

 
df3 = past_price('BTCBUSD', '30m', start_date2, end_date2)    #let the frequency of 30 min be left as a constant (do not change it, otherwise the graphs below will be messed up)
df3

#the max number of rows you get is limited to 1000        
#when you are putting in the dates, the range is limited to 21 days (assuming the frequency is 30 min)    
#for the amount of days <21 there is no limitation

In [ ]:
#here the frequency is 30 min
#this data is constant and does not change. You generate data for all the transactions that you made since the beginning

b = date.today()
a = pd.date_range(start='2021-04-26', end=b, freq='30min')    #this is to create index for a DataFrame (df5)

df5 = pd.DataFrame(a)
df5.rename(columns = {0:'Date'}, inplace = True)
df5 = df5.set_index('Date')


#money spent for BTC, not including fees
df5['Paid_in_GBP'] = np.NaN    
df5['Paid_in_GBP'].loc['2021-04-26 00:00:00'] = 300
df5['Paid_in_GBP'].loc['2021-06-01 20:00:00'] = 300
df5['Paid_in_GBP'].loc['2022-01-01 21:00:00'] = 300
df5['Paid_in_GBP'].loc['2022-06-01 20:00:00'] = 300


#conversion to USD based on https://www.xe.com/currencycharts/?from=GBP&to=USD&view=2Y
df5['GBP_to_USD_Paid'] = np.NaN
df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00'] = 300 * 1.38676     #conversion rate on 26.04.21
df5['GBP_to_USD_Paid'].loc['2021-06-01 20:00:00'] = 300 * 1.42267   
df5['GBP_to_USD_Paid'].loc['2022-01-01 21:00:00'] = 300 * 1.35289
df5['GBP_to_USD_Paid'].loc['2022-06-01 20:00:00'] = 300 * 1.26073
df5['GBP_to_USD_Paid'] = df5['GBP_to_USD_Paid'].round(2)
df5['GBP_to_USD_Paid'] = df5['GBP_to_USD_Paid'].fillna(0)


#fees from buying BTC on a crypto exchange (example)
df5['Fees_in_GBP'] = np.NaN
df5['Fees_in_GBP'].loc['2021-04-26 00:00:00'] = 5
df5['Fees_in_GBP'].loc['2021-06-01 20:00:00'] = 5
df5['Fees_in_GBP'].loc['2022-01-01 21:00:00'] = 5
df5['Fees_in_GBP'].loc['2022-06-01 20:00:00'] = 5


#conversion of fees to USD
df5['GBP_to_USD_Fees'] = np.NaN
df5['GBP_to_USD_Fees'].loc['2021-04-26 00:00:00'] = 5 * 1.38676 
df5['GBP_to_USD_Fees'].loc['2021-06-01 20:00:00'] = 5 * 1.42267
df5['GBP_to_USD_Fees'].loc['2022-01-01 21:00:00'] = 5 * 1.35289
df5['GBP_to_USD_Fees'].loc['2022-06-01 20:00:00'] = 5 * 1.26073
df5['GBP_to_USD_Fees'] = df5['GBP_to_USD_Fees'].round(2)
df5['GBP_to_USD_Fees'] = df5['GBP_to_USD_Fees'].fillna(0)

#df5.loc[start_date2 : end_date2]

In [ ]:
#here you generate BTC prices for the time intervals of the transactions made
#since with this code you can only generate a short time period you must repeat it several times but with different DataFrames

#first transaction
def past_price1(crypto,interval,start,end):    
    start = calendar.timegm(datetime.fromisoformat(start).timetuple()) * 1000   #if you have date as a string in the form of 'yyyy-mm-dd', you use this to convert the string into a unix time
    end = calendar.timegm(datetime.fromisoformat(end).timetuple()) * 1000  #calendar.timegm() returns unix time, however you need to have timetuple inside brackets (see docs)
    url= 'https://api.binance.com/api/v3/klines?symbol=' + crypto + '&interval=' + interval + '&limit=1000'
    url+= '&startTime=' + str(start)
    url+= '&endTime=' + str(end)
    df6 = pd.read_json(url)     #you take data from API (you get raw data)
    df6.columns = ['Opentime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Closetime',
                       'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore']    #to name the columns
    df6.Opentime = pd.to_datetime(df6.Opentime, unit='ms')   #conversion from unix time
    df6.Closetime = pd.to_datetime(df6.Closetime, unit='ms')
    df6['Date'] = df6.Opentime.dt.strftime("%d/%m/%Y")     #to get a specified date format
    df6['Time'] = df6.Opentime.dt.strftime("%H:%M:%S")
    df6 = df6.drop(['Closetime', 'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote',
                      'Ignore'], axis=1)
    column_names = ['Opentime', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df6 = df6[column_names]     #to have this specific sequence of columns in a DataFrame
    df6 = df6.set_index('Opentime')
    return df6
df6 = past_price1('BTCBUSD', '30m', '2021-04-26 00:00:00', '2021-04-26 02:00:00')    #let the frequency of 30 min be left as a constant (do not change it, otherwise the graphs will be messed up)


#second transaction
def past_price2(crypto,interval,start,end):    
    start = calendar.timegm(datetime.fromisoformat(start).timetuple()) * 1000   #if you have date as a string in the form of 'yyyy-mm-dd', you use this to convert the string into a unix time
    end = calendar.timegm(datetime.fromisoformat(end).timetuple()) * 1000  #calendar.timegm() returns unix time, however you need to have timetuple inside brackets (see docs)
    url= 'https://api.binance.com/api/v3/klines?symbol=' + crypto + '&interval=' + interval + '&limit=1000'
    url+= '&startTime=' + str(start)
    url+= '&endTime=' + str(end)
    df7 = pd.read_json(url)     #you take data from API (you get raw data)
    df7.columns = ['Opentime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Closetime',
                       'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore']    #to name the columns
    df7.Opentime = pd.to_datetime(df7.Opentime, unit='ms')   #conversion from unix time
    df7.Closetime = pd.to_datetime(df7.Closetime, unit='ms')
    df7['Date'] = df7.Opentime.dt.strftime("%d/%m/%Y")     #to get a specified date format
    df7['Time'] = df7.Opentime.dt.strftime("%H:%M:%S")
    df7 = df7.drop(['Closetime', 'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote',
                      'Ignore'], axis=1)
    column_names = ['Opentime', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df7 = df7[column_names]     #to have this specific sequence of columns in a DataFrame
    df7 = df7.set_index('Opentime')
    return df7
df7 = past_price2('BTCBUSD', '30m', '2021-06-01 20:00:00', '2021-06-01 23:00:00')    #let the frequency of 30 min be left as a constant (do not change it, otherwise the graphs will be messed up)


#third transaction
def past_price3(crypto,interval,start,end):    
    start = calendar.timegm(datetime.fromisoformat(start).timetuple()) * 1000   #if you have date as a string in the form of 'yyyy-mm-dd', you use this to convert the string into a unix time
    end = calendar.timegm(datetime.fromisoformat(end).timetuple()) * 1000  #calendar.timegm() returns unix time, however you need to have timetuple inside brackets (see docs)
    url= 'https://api.binance.com/api/v3/klines?symbol=' + crypto + '&interval=' + interval + '&limit=1000'
    url+= '&startTime=' + str(start)
    url+= '&endTime=' + str(end)
    df8 = pd.read_json(url)     #you take data from API (you get raw data)
    df8.columns = ['Opentime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Closetime',
                       'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore']    #to name the columns
    df8.Opentime = pd.to_datetime(df8.Opentime, unit='ms')   #conversion from unix time
    df8.Closetime = pd.to_datetime(df8.Closetime, unit='ms')
    df8['Date'] = df8.Opentime.dt.strftime("%d/%m/%Y")     #to get a specified date format
    df8['Time'] = df8.Opentime.dt.strftime("%H:%M:%S")
    df8 = df8.drop(['Closetime', 'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote',
                      'Ignore'], axis=1)
    column_names = ['Opentime', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df8 = df8[column_names]     #to have this specific sequence of columns in a DataFrame
    df8 = df8.set_index('Opentime')
    return df8
df8 = past_price3('BTCBUSD', '30m', '2022-01-01 21:00:00', '2022-01-01 23:00:00')    #let the frequency of 30 min be left as a constant (do not change it, otherwise the graphs will be messed up)


#fourth transaction
def past_price4(crypto,interval,start,end):    
    start = calendar.timegm(datetime.fromisoformat(start).timetuple()) * 1000   #if you have date as a string in the form of 'yyyy-mm-dd', you use this to convert the string into a unix time
    end = calendar.timegm(datetime.fromisoformat(end).timetuple()) * 1000  #calendar.timegm() returns unix time, however you need to have timetuple inside brackets (see docs)
    url= 'https://api.binance.com/api/v3/klines?symbol=' + crypto + '&interval=' + interval + '&limit=1000'
    url+= '&startTime=' + str(start)
    url+= '&endTime=' + str(end)
    df9 = pd.read_json(url)     #you take data from API (you get raw data)
    df9.columns = ['Opentime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Closetime',
                       'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote', 'Ignore']    #to name the columns
    df9.Opentime = pd.to_datetime(df9.Opentime, unit='ms')   #conversion from unix time
    df9.Closetime = pd.to_datetime(df9.Closetime, unit='ms')
    df9['Date'] = df9.Opentime.dt.strftime("%d/%m/%Y")     #to get a specified date format
    df9['Time'] = df9.Opentime.dt.strftime("%H:%M:%S")
    df9 = df9.drop(['Closetime', 'Quote asset volume', 'Number of trades', 'Taker by base', 'Taker buy quote',
                      'Ignore'], axis=1)
    column_names = ['Opentime', 'Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df9 = df9[column_names]     #to have this specific sequence of columns in a DataFrame
    df9 = df9.set_index('Opentime')
    return df9
df9 = past_price4('BTCBUSD', '30m', '2022-06-01 20:00:00', '2022-06-01 23:00:00')    #let the frequency of 30 min be left as a constant (do not change it, otherwise the graphs will be messed up)


In [ ]:
df4 = pd.DataFrame(df3, columns = ('Opentime','Date','Close'))      #I took "Close" price of BTC
df4.rename(columns = {'Close':'BTC_Price'}, inplace = True)
df4 = df4.set_index('Opentime')


#BTC that was owned
df4['BTC_Own'] = np.NaN
df4['BTC_Own'].loc['2021-04-26 00:00:00':'2021-06-01 19:30:00'] = df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00'] / df6['Close'].loc['2021-04-26 00:00:00']
df4['BTC_Own'].loc['2021-06-01 20:00:00':'2022-01-01 20:30:00'] = (df5['GBP_to_USD_Paid'].loc['2021-06-01 20:00:00'] / df7['Close'].loc['2021-06-01 20:00:00']) + (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00'] / df6['Close'].loc['2021-04-26 00:00:00'])
df4['BTC_Own'].loc['2022-01-01 21:00:00':'2022-06-01 19:30:00'] = (df5['GBP_to_USD_Paid'].loc['2022-01-01 21:00:00'] / df8['Close'].loc['2022-01-01 21:00:00']) + (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00'] / df6['Close'].loc['2021-04-26 00:00:00']) + (df5['GBP_to_USD_Paid'].loc['2021-06-01 20:00:00'] / df7['Close'].loc['2021-06-01 20:00:00'])
df4['BTC_Own'].loc['2022-06-01 20:00:00':df4.index[-1]] = df5['GBP_to_USD_Paid'].loc['2022-06-01 20:00:00'] / df9['Close'].loc['2022-06-01 20:00:00'] + (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00'] / df6['Close'].loc['2021-04-26 00:00:00']) + (df5['GBP_to_USD_Paid'].loc['2021-06-01 20:00:00'] / df7['Close'].loc['2021-06-01 20:00:00']) + (df5['GBP_to_USD_Paid'].loc['2022-01-01 21:00:00'] / df8['Close'].loc['2022-01-01 21:00:00'])


#actual money (USD) owned at different time periods
df4['USD_Owned'] = np.NaN
df4['USD_Owned'].loc[start_date2 : end_date2] = df4['BTC_Own'].loc[start_date2 : end_date2].multiply(df4['BTC_Price'].loc[start_date2 : end_date2])
df4['USD_Owned'] = df4['USD_Owned'].round(2)


#profit or loss of the investment 
df4['Gain_Loss_in_USD'] = np.NaN
a = calendar.timegm(datetime.fromisoformat(start_date2).timetuple())    #conversion to unix
b = time(hour=0, minute=30, second=0, microsecond=0)
c=((a - 1800))   #deduct 30 min from start_date2
d = pd.to_datetime(c, unit='s')
e = d.strftime("%Y-%m-%d %H:%M:%S")

if start_date2 == '2021-04-26 00:00:00' or '2021-06-01 20:00:00' or '2022-01-01 21:00:00' or '2022-06-01 20:00:00':
    df4['Gain_Loss_in_USD'].loc[start_date2 : end_date2] = df4['USD_Owned'].loc[start_date2 : end_date2] - (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00' : e].sum() + df5['GBP_to_USD_Paid'].loc[start_date2 : end_date2].cumsum() + df5['GBP_to_USD_Fees'].loc['2021-04-26 00:00:00' : e].sum() + df5['GBP_to_USD_Fees'].loc[start_date2 : end_date2].cumsum()) 
else:
    df4['Gain_Loss_in_USD'].loc[start_date2 : end_date2] = df4['USD_Owned'].loc[start_date2 : end_date2] - (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00' : start_date2].sum() + df5['GBP_to_USD_Paid'].loc[start_date2 : end_date2].cumsum() + df5['GBP_to_USD_Fees'].loc['2021-04-26 00:00:00' : start_date2].sum() + df5['GBP_to_USD_Fees'].loc[start_date2 : end_date2].cumsum())

    
#percentage of loss/profit compared to the investment
df4['Gain_Loss_Pct.'] = np.NaN

if start_date2 == '2021-04-26 00:00:00' or '2021-06-01 20:00:00' or '2022-01-01 21:00:00' or '2022-06-01 20:00:00':
    df4['Gain_Loss_Pct.'].loc[start_date2 : end_date2] = df4['USD_Owned'].loc[start_date2 : end_date2] / (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00' : e].sum() + df5['GBP_to_USD_Paid'].loc[start_date2 : end_date2].cumsum() + df5['GBP_to_USD_Fees'].loc['2021-04-26 00:00:00' : e].sum() + df5['GBP_to_USD_Fees'].loc[start_date2 : end_date2].cumsum()) 
else:
    df4['Gain_Loss_Pct.'].loc[start_date2 : end_date2] = df4['USD_Owned'].loc[start_date2 : end_date2] / (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00' : start_date2].sum() + df5['GBP_to_USD_Paid'].loc[start_date2 : end_date2].cumsum() + df5['GBP_to_USD_Fees'].loc['2021-04-26 00:00:00' : start_date2].sum() + df5['GBP_to_USD_Fees'].loc[start_date2 : end_date2].cumsum())
df4['Gain_Loss_Pct.'] = df4['Gain_Loss_Pct.'].round(4)


#the percentage change between values in a column
df4['Pct. Change (Gain/Loss)'] = np.NaN
df4['Pct. Change (Gain/Loss)'].loc[start_date2 : end_date2] = df4['Gain_Loss_in_USD'].loc[start_date2 : end_date2].pct_change() 


#change between values in a column in USD
df4['USD Change (Gain/Loss)']=df4['Pct. Change (Gain/Loss)']
df4['USD Change (Gain/Loss)']=df4['USD Change (Gain/Loss)'].shift(periods=-1, axis='index')
df4['USD Change (Gain/Loss)'].loc[start_date2 : end_date2]=df4['Gain_Loss_in_USD'].loc[start_date2 : end_date2].multiply(df4['USD Change (Gain/Loss)'].loc[start_date2 : end_date2])
df4['USD Change (Gain/Loss)']=df4['USD Change (Gain/Loss)'].shift(periods=1, axis='index')
df4['USD Change (Gain/Loss)'] = df4['USD Change (Gain/Loss)'].round(2)


#BTC price that you need to be even 
df4['BTC_Price_Even'] = np.NaN

if start_date2 == '2021-04-26 00:00:00' or '2021-06-01 20:00:00' or '2022-01-01 21:00:00' or '2022-06-01 20:00:00':
    df4['BTC_Price_Even'].loc[start_date2 : end_date2] = (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00' : e].sum() + df5['GBP_to_USD_Paid'].loc[start_date2 : end_date2].cumsum() + df5['GBP_to_USD_Fees'].loc['2021-04-26 00:00:00' : e].sum() + df5['GBP_to_USD_Fees'].loc[start_date2 : end_date2].cumsum()) / df4['BTC_Own'].loc[start_date2 : end_date2]
else:
    df4['BTC_Price_Even'].loc[start_date2 : end_date2] = (df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00' : start_date2].sum() + df5['GBP_to_USD_Paid'].loc[start_date2 : end_date2].cumsum() + df5['GBP_to_USD_Fees'].loc['2021-04-26 00:00:00' : start_date2].sum() + df5['GBP_to_USD_Fees'].loc[start_date2 : end_date2].cumsum()) / df4['BTC_Own'].loc[start_date2 : end_date2]

    

#cumulative return of BTC       https://www.investopedia.com/terms/c/cumulativereturn.asp
df4['Cum_Ret_BTC'] = np.NaN  
df4['Cum_Ret_BTC'] = ((df4['BTC_Price'].pct_change()) + 1).cumprod() - 1 


df4.loc[start_date2 : end_date2]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))


if len(df3['Opentime'])<=219:    
    date_form = DateFormatter('%H-%M')
    ax1.set(xlabel='Time (h)', ylabel='BTC Price (USD)')
    ax1.set_title('BTC Price from {} to {} - y-m-d'.format(start_date, end_date))
    ax2.set(xlabel='Time (h)', ylabel='Amount (USD)')
    ax3.set(xlabel='Time (h)', ylabel='Amount (USD)')
else:
    date_form = DateFormatter('%d-%m')
    ax1.set(xlabel='Date (Day-Month)', ylabel='BTC Price (USD)')
    ax1.set_title('BTC Price ')
    ax2.set(xlabel='Date (Day-Month)', ylabel='Amount (USD)')
    ax3.set(xlabel='Date (Day-Month)', ylabel='Amount (USD)')


ax1.plot(df3['Opentime'], df3['Close'], color='xkcd:black')    #https://matplotlib.org/stable/tutorials/colors/colors.html
#https://xkcd.com/color/rgb/
ax1.xaxis.label.set_size(12)
ax1.yaxis.label.set_size(12)
ax1.xaxis.set_major_formatter(date_form)
t = df3['Close'].max()
t2 = df3['Close'].min()
ax1.axhline(y=t, color='xkcd:dusty rose', linestyle='--')
ax1.axhline(y=t2, color='xkcd:light olive', linestyle='--')

ax2.plot(df3['Opentime'], df4['Gain_Loss_in_USD'], color='xkcd:blue')        #you can change here to ax2.scatter to get other types, etc.
ax2.set_title('Profit or Loss ($)')
ax2.xaxis.label.set_size(12)
ax2.yaxis.label.set_size(12)
ax2.xaxis.set_major_formatter(date_form)
m = df4['Gain_Loss_in_USD'].max()
m2 = df4['Gain_Loss_in_USD'].min()
ax2.axhline(y=m, color='xkcd:green', linestyle='--')
ax2.axhline(y=m2, color='xkcd:red', linestyle='--')


ax3.plot(df3['Opentime'], df4['USD_Owned'], color='xkcd:orange')
ax3.set_title('Actual $ Owned')
ax3.xaxis.label.set_size(12)
ax3.yaxis.label.set_size(12)
ax3.xaxis.set_major_formatter(date_form)
m1 = df4['USD_Owned'].max()
ax3.axhline(y=m1, color='xkcd:violet', linestyle='--')


sum_fees = df5['GBP_to_USD_Fees'].loc['2021-04-26 00:00:00':end_date2].sum()
sum_paid = df5['GBP_to_USD_Paid'].loc['2021-04-26 00:00:00':end_date2].sum()
def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return "{:.1f}%\n({:d} $)".format(pct, absolute)
wedges, texts, autotexts = ax4.pie([sum_fees, sum_paid], autopct=lambda pct: func(pct, [sum_fees, sum_paid]), 
                                   colors=['xkcd:royal blue','xkcd:dark green'], textprops=dict(color="xkcd:white"))
ax4.legend(wedges, ['Fees Paid ($)', 'Amount Invested ($)'], title="Legend", loc="center left", bbox_to_anchor=(0.9, 0, 0.5, 1))
plt.setp(autotexts, size=10, weight="bold")
plt.show()


In [ ]:
fig, ((ax5, ax6), (ax7, ax8)) = plt.subplots(2, 2, figsize=(15, 12))

if len(df3['Opentime'])<=219:    
    date_form = DateFormatter('%H-%M')
    ax5.set(xlabel='Time (h)', ylabel='Amount (USD)')
    ax5.set_title('Volatility of profit or loss ($) every 30 min from {} to {} - y-m-d'.format(start_date, end_date))
    ax7.set(xlabel='Time (h)')
else:
    date_form = DateFormatter('%d-%m')
    ax5.set(xlabel='Date (Day-Month)', ylabel='Amount (USD)')
    ax5.set_title('Volatility of profit or loss ($) every 30 min')
    ax7.set(xlabel='Date (Day-Month)')



#profit or loss in USD over time
ax5.plot(df3['Opentime'],df4['USD Change (Gain/Loss)'], color='xkcd:dark brown')       
ax5.xaxis.label.set_size(12)
ax5.yaxis.label.set_size(12)
ax5.xaxis.set_major_formatter(date_form)
u = df4['USD Change (Gain/Loss)'].max()
l = df4['USD Change (Gain/Loss)'].min()
ax5.axhline(y=u, color='xkcd:mint', linestyle='--')
ax5.axhline(y=l, color='xkcd:pastel green', linestyle='--')


#histogram of profit or loss in USD over time
ax6.hist(df4['USD Change (Gain/Loss)'], rwidth=0.5, bins=1000, linewidth=0.5, edgecolor='xkcd:army green')  
ax6.set_title('Distribution of profit or loss ($) every 30 min')
ax6.set(xlabel='USD', ylabel='Frequency')
ax6.xaxis.label.set_size(12)
ax6.yaxis.label.set_size(12)


#the price that BTC had to hit at particular time to make the investment even
ax7.plot(df3['Opentime'], df3['Close'], color='xkcd:green')
ax7.set_ylim(15000,71000)
ax7.set_title('Actual BTC Price vs Price to be Even')
ax7.set_ylabel('BTC Price (USD)')
ax7.xaxis.label.set_size(12)
ax7.yaxis.label.set_size(12)
ax = ax7.twinx()                              #https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html#sphx-glr-gallery-subplots-axes-and-figures-two-scales-py
ax.plot(df3['Opentime'], df4['BTC_Price_Even'], color='xkcd:mauve')
ax.set_ylim(15000,71000)
ax.set_ylabel('Hypothetical BTC Price (USD)')
ax.yaxis.label.set_size(12)
ax.legend(['Hypothetical Price'])
fig.tight_layout()


#the percentage of time in gain vs in loss

gain=[]
loss=[]
even=[]
for v in df4['Gain_Loss_in_USD']:
    if v>0:
        gain.append('a')
    elif v<0:
        loss.append('b')
    elif v==0:
        even.append('c')
    else:
        pass
    
a = len(gain) 
b = len(loss)
c = len(even)
d = a + b + c
g = a/d
l = b/d

def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return "{:.0f}%".format(pct, absolute)
wedges, texts, autotexts = ax8.pie([g, l], autopct=lambda pct: func(pct, [g, l]), 
                                   colors=['xkcd:burnt orange','xkcd:dark red'], textprops=dict(color="xkcd:black"))
ax8.legend(wedges, ['Time pct. in profit', 'Time pct. in loss'], title="Legend", loc="center left", bbox_to_anchor=(0.9, 0, 0.5, 1))
plt.setp(autotexts, size=10, weight="bold")
ax8.set_title('Percentage of time the investment was/is in profit/loss')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15, 5))

#date_form = DateFormatter('%y-%m')

if len(df3['Opentime'])<=219:    
    date_form = DateFormatter('%H-%M')
    ax1.set(xlabel='Time (h)')
else:
    date_form = DateFormatter('%d-%m')
    ax1.set(xlabel='Date (Day-Month)')

ax1.set_title('Profit/loss in pct. for a total investment')
ax1.plot(df3['Opentime'],df4['Gain_Loss_Pct.'], 'xkcd:greyish blue')
ax1.xaxis.label.set_size(12)
ax1.yaxis.label.set_size(12)
ax1.xaxis.set_major_formatter(date_form)
p = df4['Gain_Loss_Pct.'].max()
p2 = df4['Gain_Loss_Pct.'].min()
ax1.axhline(y=p, color='xkcd:brick', linestyle='--')
ax1.axhline(y=p2, color='xkcd:ocean blue', linestyle='--')


#https://sixsigmastudyguide.com/box-plots/
ax2.boxplot(df4['Gain_Loss_Pct.'])
ax2.set_title('Boxplot of profit/loss in pct. for a total investment')
ax2.yaxis.grid(True, linestyle='-', which='major', color='xkcd:light grey',
               alpha=0.5)
ax2.set_xticks(ticks=[1], labels=['y=1 (even), y<1 (loss), y>1 (profit)'])
ax2.xaxis.label.set_size(12)


In [ ]:
#cum return of BTC

#date_form = DateFormatter('%y-%m')

fig, ax = plt.subplots(figsize=(8, 5))

if len(df3['Opentime'])<=219:    
    date_form = DateFormatter('%H-%M')
    ax.set(xlabel='Time (h)')
    ax.set_title('Cumulative return of BTC from {} to {} - y-m-d'.format(start_date, end_date))
else:
    date_form = DateFormatter('%d-%m')
    ax.set(xlabel='Date (Day-Month)')
    ax.set_title('Cumulative return of BTC from {} to {} - y-m-d'.format(start_date, end_date))

ax.plot(df3['Opentime'],df4['Cum_Ret_BTC'], 'xkcd:light olive')
ax.xaxis.label.set_size(12)
ax.yaxis.label.set_size(12)
ax.xaxis.set_major_formatter(date_form)
i = df4['Cum_Ret_BTC'].max()
i2 = df4['Cum_Ret_BTC'].min()
ax.axhline(y=i, color='xkcd:greenish yellow', linestyle='--')
ax.axhline(y=i2, color='xkcd:yellowish green', linestyle='--')

plt.show()

In [ ]:
abc=df4.groupby('Gain_Loss_in_USD')

def peak_to_peak(arr):
    return arr.max()

abc.agg(peak_to_peak)


In [ ]:
abc=df4.groupby('USD_Owned')

def peak_to_peak(arr):
    return arr.max()

abc.agg(peak_to_peak)